# Final Report

## Introduction
The dataset contains 32 columns. Each column corresponds to a specific attribute for each participant. The values in all columns were originally categorical but have been quantified to real numbers in the dataset. The level of consumption, however, for each drug is categorical and contains the following classes:

- CLO: Never used
- CL1: Used over a decade ago
- CL2: Used in last decade
- CL3: Used in last year
- CL4: Used in last month
- CL5: Used in last week
- CL6: Used in last day

Among all the legal and illegal drugs in the dataset, there is a fictitious drug called Semeron which was used in the survey to identiy over-claimers. Here is information regarding all the columns:

- ID 
- Age (18-65+)
- Gender
- Education (level of education)
- Country (country of residence)
- Ethnicity (ethnicity)
- Nscore (measurement of neuroticism)
    + 12 to 60 (actual)
    + -3.46436 to 3.27393 (quantified)
- Escore (measurement of extraversion) 
    + 16 to 59 (actual)
    + -3.27393 to 3.27393 (quantified)
- Oscore (openness to experience)
    + 24 to 60 (actual)
    + -3.27393 to 2.90161 (quantified)
- Ascore (measurement of agreeableness)
    + 12 to 60 (actual)
    + -3.46436 to 3.46436 (quantified)
- Cscore (measurement of conscientiousness)
    + 17 to 59 (actual)
    + -3.46436 to 3.46436 (quantified)
- IScore (measurement of impulsiveness)
    + no actual values were provided
    + -2.55524-2.90161 (quantified)
- SS (measurement of sensation seeking)
    + no actual values were provided
    + -2.07848-1.92173 (quantified)
- Alcohol (level alcohol consumption)
- Amphet (level of amphetamines consumption)
- Amyl (level of amyl nitrite consumption)
- Benzos (level of benzodiazepine consumption)
- Caff (level of caffeine consumption)
- Cannabis (level of cannabis consumption)
- Choc (level of chocolate consumption)
- Coke (level of cocaine consumption)
- Crack (level of crack consumption)
- Ecstasy(level of ecstasy consumption)
- Heroin (level of heroin consumption)
- Ketamine (level of ketamine consumption)
- Legalh (level of legal highs consumption) 
- LSD is (level of LSD consumption)
- Meth (level of methadone consumption) 
- Mushrooms (level of magic mushrooms consumption) 
- Nicotine (level of nicotine consumption)
- Semer (level of fictitious drug Semeron consumption) 
- VSA (level of volatile substance abuse) 

## Methods and Results
### Loading libraries

In [20]:
library(tidyverse)
library(testthat)
library(digest)
library(repr)
library(tidymodels)
library(GGally)
options(repr.matrix.max.rows = 7) # limits the outputs of data frames to 7 rows (or observations)

### Reading the data
We will read the drugs consumption quantified dataset directly from the URL and save it to a dataframe called "drugs_data". Since the columns in the dataset have no headings, we will set `col_names` to `FALSE`.

In [21]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data"
drugs_data <- read_csv(url, col_names = FALSE)
drugs_data

Rows: 1885 Columns: 32
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (19): X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, X...
dbl (13): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,⋯,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,⋯,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,⋯,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,⋯,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1886,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,⋯,CL4,CL0,CL2,CL0,CL2,CL0,CL2,CL6,CL0,CL0
1887,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,⋯,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL4,CL0,CL0
1888,-0.95197,-0.48246,-0.61113,0.21128,-0.31685,-0.46725,2.12700,1.65653,1.11406,⋯,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL6,CL0,CL2


### Wrangling and cleaning the dataset
- We do not need to check for any missing values in the dataset as it is mentioned in the website that there are not any.
- We will rename the column headings to suitable names.
- Each row in the data set corresponds to a single participant (observation), each column to a single variable (attribute) and each cell to a single value. Thus, our dataset is in tidy format.
- We will select only the columns that give us information about the participant's age, personality measurements, level of alcohol and semeron consumption as these are the only variables from the dataset that we will be needing for our planned analysis.
- Since there are observations where the consumption for the fictitious drug `Semer` is not `CLO` (never used), we will filter out these observations as they are not useful and then remove this column as well.
- We will use `mutate` and `as_factor` functions to ensure that our response variable `Alcohol` is treated as factor which is neccesary for our classficiation model.

In [31]:
colnames(drugs_data) <- c("ID", "Age", "Gender", "Education", "Country", "Ethnicity", "NScore", "EScore", "OScore", "AScore", 
                          "CScore", "IScore", "SS", "Alcohol", "Amphet", "Amyl", "Benzos", "Caff", "Cannabis", "Choc", "Coke",
                          "Crack", "Ecstacy", "Heorin", "LegalH", "LSD", "Meth", "Mushrooms", "Nicotine", "Semer", "VSA")

alcohol_data <- select(drugs_data, Age, NScore , EScore, OScore, AScore,
                                IScore, CScore, SS, Semer, Alcohol)

alcohol_data_filtered <- alcohol_data |>
                         filter(Semer == "CL0") |>
                         select(-Semer)

alcohol_clean <- alcohol_data_filtered |>
                 mutate(Alcohol = as_factor(Alcohol))
alcohol_clean

Age,NScore,EScore,OScore,AScore,IScore,CScore,SS,Alcohol
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0.49788,-0.46725,0.80523,-0.84732,-1.62090,-1.37983,-1.01450,0.40148,CL6
0.49788,-1.32828,1.93886,-0.84732,-0.30172,0.19268,1.63088,-0.52593,CL5
1.82213,-2.05048,-1.50796,-1.55521,-1.07533,-0.71126,1.13407,-0.52593,CL5
1.82213,1.37297,-0.15487,-0.17779,-1.92595,-0.71126,-1.51840,-0.21575,CL6
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
-0.95197,-0.79151,0.00332,2.44904,0.76096,0.88113,-1.51840,1.92173,CL5
2.59171,2.12700,-0.15487,2.44904,0.94156,-0.21712,-0.65253,0.07987,CL5
-0.95197,-1.19430,1.74091,1.88511,0.76096,0.88113,-1.13788,1.92173,CL5


### Exploratory data analysis
After cleaning our dataset, we are left with 428 observations. From this point on, we will be using 75% of our data set and we will use the rest 25% only to test the accuracy of our classifier. Thus, we will split our dataset into training and testing sets using the `initial_split` function.

In [33]:
set.seed(69)
alcohol_split <- initial_split(alcohol_data_filtered, prop = 0.75, strata = Alcohol)
alcohol_train <- training(alcohol_split)
alcohol_test <- testing(alcohol_split)

#### Summary of the dataset
We will summarize our dataset in two ways:
- A table that reports the number of observations for each class in `Alcohol`
- A table that reports the mean of all personality measurements grouped by the `Alcohol` variable.

In [36]:
alcohol_count <- group_by(alcohol_train, Alcohol) |>
                 summarize(n = n())
alcohol_count

Alcohol,n
<chr>,<int>
CL0,16
CL1,6
CL2,11
CL3,35
CL4,48
CL5,137
CL6,66


In [37]:
alcohol_means <- alcohol_train |>
                 group_by(Alcohol) |>
                 summarize(NScore_Mean = mean(NScore),
                           EScore_Mean = mean(EScore),
                           OScore_Mean = mean(OScore),
                           AScore_Mean = mean(AScore),
                           IScore_Mean = mean(IScore),
                           CScore_Mean = mean(CScore))
alcohol_means

Alcohol,NScore_Mean,EScore_Mean,OScore_Mean,AScore_Mean,IScore_Mean,CScore_Mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CL0,0.1190381,0.1158106250,0.0469375,0.02410312,-0.3413656,0.23926312
CL1,-0.5894517,0.4553266667,-0.9030767,0.75377667,-0.2583617,1.11948667
CL2,0.5474755,-0.5713127273,-0.6111855,-0.25231273,-0.4266245,0.02889545
CL3,-0.2083551,-0.1460434286,-0.4620551,-0.05292543,-0.4505683,-0.13884057
CL4,-0.2381323,-0.0304225000,-0.3216004,0.37365021,-0.2580952,0.35648354
CL5,-0.2104826,0.1737224088,-0.3266285,0.33174526,-0.3653848,0.39946504
CL6,-0.1070455,0.0003836364,-0.2337532,0.20108061,-0.2593374,0.33426621
